<a href="https://colab.research.google.com/github/ymiftah/operations_research/blob/master/transportation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q pyomo
!apt-get install -qq coinor-cbc

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

# A Transportation Problem

## Problem Statement
An Italian transportation ﬁrm should carry some empty containers from its 6 stores (in Verona, Perugia, Rome, Pescara, Taranto and Lamezia) to the main national ports (Genoa, Venice, Ancona, Naples, Bari). The container stocks at the stores are the following:

|         | Empty Containers   |
|---------|-------------------:|
|Verona   | 10|
|Perugia  | 12|
|Rome     | 20|
|Pescara  | 24|
|Taranto  | 18|
|Lamezia  | 40|

The demands at the ports are as follows:

|         | Container demand   |
|---------|-------------------:|
|Genoa   | 20|
|Venice  | 15|
|Ancona     | 25|
|Naples  | 33|
|Bari  | 21|

Transportation is carried out by a ﬂeet of lorries. The transportation cost for each container is proportional to the distance travelled by the lorry, and amounts to 30 euro / km. Every lorry can carry at most 2 containers. Distances are as follows:

|[km]    | Genoa | Venice | Ancona | Naples | Bari
|----|----:|----:|----:|----:|----:|
|Verona| 290|115|355|715|810|
|Perugia | 380|340|165|380|610|
|Rome|505|530|285|220|450|
|Pescara|655|450|155|240|315|
|Taranto|1010|840|550|305|95|
|Lamezia|1072|1097|747|372|333|

**Find the policy minimizing transportation cost total revenue**

In [0]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

stores  =   ['Verona', 'Perugia', 'Rome', 'Pescara', 'Taranto', 'Lamezia']
empty       = dict(zip(stores, [10, 12, 20, 24, 18, 40]))

ports  = ['Genoa', 'Venice', 'Ancona', 'Naples', 'Bari']
demand = dict(zip(ports, [20, 15, 25, 33, 21]))

array = [[290, 115, 355, 715, 810],
         [380, 340, 165, 380, 610],
         [505, 530, 285, 220, 450],
         [655, 450, 155, 240, 315],
         [1010, 840, 550, 305, 95],
         [1072, 1097, 747, 372, 333]]
distance = {(stores[i], ports[j]): array[i][j]
            for i in range(len(stores))
            for j in range(len(ports))}

cost = 30 # Eur/km

# Sets
model.stores = Set(initialize=stores)
model.ports = Set(initialize=ports)

# declare decision variables
model.lorries = Var(model.stores, model.ports, domain=NonNegativeIntegers)
model.ship = Var(model.stores, model.ports, domain=NonNegativeIntegers)

# declare objective
model.cost = Objective(expr = cost * sum(model.lorries[i,p] * distance[i,p]
                                    for i in model.stores
                                    for p in ports),
                         sense=minimize)

#declare constraints
def cstr_demand(m, p):
    return sum(m.ship[i, p] for i in m.stores) == demand[p]

def cstr_stock(m, i):
    return sum(m.ship[i, p] for p in m.ports) <= empty[i]

def cstr_lorry(m, i, p):
    return m.lorries[i, p] >= m.ship[i,p] / 2


model.demand = Constraint(model.ports, rule=cstr_demand)
model.stock = Constraint(model.stores, rule=cstr_stock)
model.capacity = Constraint(model.stores, model.ports, rule=cstr_lorry)

In [48]:
# Solve the problem
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model)

print('Cost = ', model.cost())

Cost =  468510.0


## Results

Let us visualize in a table the shipping policy obtained

In [49]:
shipping = model.ship.extract_values()

df = pd.DataFrame(
    [[shipping[i,j]for j in ports] for i in stores],
    columns=ports,
    index=stores,
    dtype='int'
)
print(df)

         Genoa  Venice  Ancona  Naples  Bari
Verona       0      10       0       0     0
Perugia      6       5       1       0     0
Rome        14       0       0       6     0
Pescara      0       0      24       0     0
Taranto      0       0       0       1    17
Lamezia      0       0       0      26     4


In [50]:
print('Containers shipped to each port:')
print(df.sum(axis=0))

Containers shipped to each port:
Genoa     20
Venice    15
Ancona    25
Naples    33
Bari      21
dtype: int64


As we see above the demand is correctly served

In [51]:
print('Containers shipped from:')
print(df.sum(axis=1))

Containers shipped from:
Verona     10
Perugia    12
Rome       20
Pescara    24
Taranto    18
Lamezia    30
dtype: int64


And the policy is within the stocks at each store